聚类

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
k=5
kmeans=KMeans(n_clusters=k)
y_pred=kmeans.fit_predict(X)
X_new = np.array([[0, 2], [3, 2], [-3, 3], [-3, 2.5]])
kmeans.transform(X_new)

In [ ]:
#中心点初始化方法
from sklearn.cluster import KMeans
import numpy as np
good_init=np.array([[-3, 3], [-3, 2], [-3, 1], [-1, 2], [0, 2]])
kmeans=KMeans(n_clusters=5, init=good_init, n_init=1)
#惯性 kmeans.inertia_
#负惯性 kmeans.score

惯性：对于$K$个簇，其中$C_{i}$是第$i$个簇，$x_{j}$是簇$C_{i}$中的一个点，$\mu_{i}$是簇$C_{i}$的质心，则惯性$I$定义为$I=\sum_{i=1}^{k}\sum_{x_{j}\in C_{i}}||x_{j}-\mu_{i}||^2$

K-Means算法的改进：K-Means++

step1. 取一个中心点$c^{(1)}$, 从数据集中随机选择一个中心点。

step2. 取一个新中心点$c^{(i)}$, 选择一个概率为$\frac{D(x^{(i)})^{2}}{\sum^{m}_{j=1}D(x^{(j)})^{2}}$的实例$x^{(i)}$与已经选择的最远中心点的距离。这种概率分布确保距离已选择的中心点较远的实例有更大可能性被选择为中心点

step3. 重复上一步，直到选择了所有中心点

In [ ]:
#小批量K-Means
from sklearn.cluster import MiniBatchKMeans
minibatch_kmeans=MiniBatchKMeans(n_clusters=5)
minibatch_kmeans.fit(X)

In [ ]:
#轮廓分数
from sklearn.metrics import silhouette_score
silhouette_score(X, kmeans.labels_)

In [ ]:
#使用聚类进行图像分割
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
import os

# 读取图像
image_path = r"img.png"
if os.path.exists(image_path):
    # 使用matplotlib读取
    from matplotlib.image import imread
    image = imread(image_path)
    print(f"图像形状: {image.shape}")
    print(f"数据类型: {image.dtype}")
    print(f"值范围: [{image.min():.3f}, {image.max():.3f}]")
else:
    print(f"文件不存在: {image_path}")
    exit()
# 关键修复：matplotlib.imread()读取PNG返回的是0-1范围的浮点数
# 但KMeans需要0-255范围的数据
if image.dtype == np.float32 or image.dtype == np.float64:
    print("检测到浮点数图像，将转换为0-255范围的整数")
    # 将0-1的浮点数转换为0-255的整数
    image_uint8 = (image * 255).astype(np.uint8)
else:
    image_uint8 = image.astype(np.uint8) if image.dtype != np.uint8 else image
# 使用KMeans聚类
X = image_uint8.reshape((-1, 3)).astype(np.float64)
kmeans = KMeans(n_clusters=8, random_state=42, n_init=10)
kmeans.fit(X)

# 创建分割后的图像
segmented_img = kmeans.cluster_centers_[kmeans.labels_]
segmented_img = segmented_img.reshape(image_uint8.shape)

# 确保值在0-255范围内并转换为uint8
segmented_img = np.clip(segmented_img, 0, 255)
segmented_img_uint8 = segmented_img.astype(np.uint8)

# 显示图像
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# 显示原始图像（确保正确显示）

axes[0].imshow(image_uint8)
axes[0].set_title('Original Image')
axes[0].axis('off')

# 显示分割后的图像
axes[1].imshow(segmented_img_uint8)
axes[1].set_title('Segmented Image (K-Means, K=8)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

# 可选：保存分割后的图像
plt.imsave('segmented_image.png', segmented_img_uint8)
print("分割完成！")

In [29]:
#使用聚类进行预处理
from sklearn.datasets import load_digits
X_digits, y_digits = load_digits(return_X_y=True)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits)
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(max_iter=1000) # 尝试 1000 次迭代
log_reg.fit(X_train, y_train)
score = log_reg.score(X_test, y_test)
print(f"Score: {score:.4f}")

Score: 0.9667


In [ ]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ("kmeans", KMeans(n_clusters=50)),
    ("log_reg", LogisticRegression()),
])
pipeline.fit(X_train, y_train)
pipeline.score(X_test, y_test)

In [ ]:
#寻找集群的最优解
from sklearn.model_selection import GridSearchCV
param_grid = dict(kmeans__n_clusters=range(2,100))
grid_clf=GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_clf.fit(X_train, y_train)
grid_clf.best_params_

In [ ]:
#使用聚类进行半监督学习
n_labeled=50
log_reg=LogisticRegression()
log_reg.fit(X_train[:n_labeled], y_train[:n_labeled])
k=50
kmeans=KMeans(n_clusters=k)
X_digits_dist = kmeans.fit_transform(X_train)
representative_digit_idx = np.argmin(X_digits_dist, axis=0)
X_representative_digits = X_train[representative_digit_idx]
y_representative_digits = np.array([4, 8, 0, 6, 8, 3, 7, 6, 2, 3, 1, 1])
y_train_propagated=np.empty(len(X_train), dtype=np.int32)
for i in range(k):
    y_train_propagated[kmeans.labels_==i] = y_representative_digits[i]

percentile_closest = 20
X_cluster_dist = X_digits_dist[np.arange(len(X_train)), kmeans.labels_]
for i in range(k):
    in_cluster = (kmeans.labels_ == i)
    cluster_dist = X_cluster_dist[in_cluster]
    cutoff_distance = np.percentile(cluster_dist, percentile_closest)
    above_cutoff = (X_cluster_dist > cutoff_distance)
    X_cluster_dist[in_cluster & above_cutoff] = -1
partially_propagated = (X_cluster_dist != -1)
X_train_partially_propagated = X_train[partially_propagated]
y_train_partially_propagated = y_train_propagated[partially_propagated]

主动学习:
1. 该模型是在目前为止收集到的带标签的实例上进行训练，并且该模型用于对所有未标记实例进行预测。
2. 把模型预测最不确定的实例（当估计的概率最低时）提供给专家来做标记。
3. 重复此过程，直到性能改进到不值得做标记为止。

DBSCAN：该算法将集群定义为高密度的连续区域。下面是它的工作原理：
1. 对于每个实例，该算法都会计算在距它一小段距离ε内有多少个实例。该区域称为实例的ε-邻域。
2. 如果一个实例在其ε邻域中至少包含min sam_ples个实例（包括自身），则该实例被视为核心实例。换句话说，核心实例是位于密集区域中的实例。
3. 核心实例附近的所有实例都属于同一集群。这个邻域可能包括其他核心实例。因此，一长串相邻的核心实例形成一个集群。
4. 任何不是核心实例且邻居中没有实例的实例都被视为异常。

如果所有集群足够密集并且被低密度区域很好地分隔开，则该算法
效果很好。

DBSCAN类没有predict()方法，无法预测新实例属于哪个集群。

DBSCAN能够识别任何数量的任何形状的集群，对异常值具有鲁棒性；如果密度在整个集群中变化很大，则可能无法正确识别所有的集群。计算复杂度大约为$O(mlogm)$

In [39]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons
X, y= make_moons(n_samples=1000, noise=0.05)
dbscan=DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X)
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=50)
knn.fit(dbscan.components_, dbscan.labels_[dbscan.core_sample_indices_])
X_new = np.array([[-0.5, 0], [0, 0.5], [1, -0.1], [2, 1]])
knn.predict(X_new)

array([6, 1, 4, 2])

从$q(z)$到$p(z|X)$的KL散度

\begin{align*}
D_{\text{KL}}(q \parallel p) &= \mathbb{E}_q\left[ \log \frac{q(z)}{p(z \mid X)} \right] \\
&= \mathbb{E}_q[\log q(z) - \log p(z \mid X)] \\
&= \mathbb{E}_q\left[ \log q(z) - \log \frac{p(z \mid X)}{p(X)} \right] \\
&= \mathbb{E}_q[\log q(z) - \log p(z \mid X) + \log p(X)] \\
&= \mathbb{E}_q[\log q(z)] - \mathbb{E}_q[\log p(z \mid X)] + \mathbb{E}_q[\log p(X)] \\
&= \mathbb{E}_q[\log p(X)] - (\mathbb{E}_q[\log p(z \mid X)] - \mathbb{E}_q[\log q(z)]) \\
&= \log p(X) - \text{ELBO} \\
\text{其中 } \text{ELBO} &= \mathbb{E}_q[\log p(z, X)] - \mathbb{E}_q[\log q(z)]
\end{align*}